In [1]:
# @title CELL 1: Investigator Input Configuration (names, lookback, time window)
# ✅ CELL 1: Investigator Input Configuration (names, lookback, time window)
import re
from datetime import datetime, timedelta, timezone

# Get input from investigator
raw_input_names = input("🎙️ Enter voice name keywords (comma-separated):\n")
lookback_days_input = input("🗓️  How many days back should we look? (e.g. 5):\n")
tts_window_minutes_input = input("⏱️  How many minutes after NGV attempt should we look for TTS generations? (e.g. 30):\n")

# Normalize and sanitize names
voice_fragments = [
    name.strip().lower().replace("’", "'") for name in raw_input_names.split(",") if name.strip()
]
escaped_fragments = [re.escape(name) for name in voice_fragments]

# Parse numeric inputs
try:
    lookback_days = int(lookback_days_input.strip())
except ValueError:
    raise ValueError("Invalid number for lookback days.")

try:
    tts_window_minutes = int(tts_window_minutes_input.strip())
except ValueError:
    raise ValueError("Invalid number for TTS time window.")

# Set up config
INVESTIGATION_CONFIG = {
    "name": "NGV Abuse Check",
    "voice_names": voice_fragments,
    "voice_names_escaped": escaped_fragments,
    "lookback_days": lookback_days,
    "tts_window_minutes": tts_window_minutes,
    "start_time": (datetime.now(timezone.utc) - timedelta(days=lookback_days)).replace(microsecond=0).isoformat(),
}

# Confirm config
print("\n✅ Investigation configuration set:")
print(f"• Name: {INVESTIGATION_CONFIG['name']}")
print(f"• Voice Keywords: {INVESTIGATION_CONFIG['voice_names']}")
print(f"• Lookback Days: {INVESTIGATION_CONFIG['lookback_days']}")
print(f"• TTS Window (minutes): {INVESTIGATION_CONFIG['tts_window_minutes']}")
print(f"• Start Time: {INVESTIGATION_CONFIG['start_time']}")


🎙️ Enter voice name keywords (comma-separated):
obama
🗓️  How many days back should we look? (e.g. 5):
7
⏱️  How many minutes after NGV attempt should we look for TTS generations? (e.g. 30):
20

✅ Investigation configuration set:
• Name: NGV Abuse Check
• Voice Keywords: ['obama']
• Lookback Days: 7
• TTS Window (minutes): 20
• Start Time: 2025-07-20T21:19:52+00:00


In [2]:
# @title Cell 1.5: BigQuery Client Setup
# ✅ Cell 1: BigQuery Client Setup
from google.cloud import bigquery

try:
    client = bigquery.Client()
    print("✅ BigQuery client initialized.")
except Exception as e:
    raise RuntimeError(f"❌ Failed to initialize BigQuery client: {e}")


✅ BigQuery client initialized.


In [3]:
# @title Cell 2: Pull NGV attempts from BigQuery (with safe substring filtering)
from datetime import datetime, timedelta
from google.cloud import bigquery

def get_ngv_attempts(config, client):
    """
    Pull NGV cloning attempts from BigQuery filtered by timestamp and voice name substrings.
    """

    start_time = config["start_time"]
    lookback_days = config["lookback_days"]
    substrings = config["voice_names"]

    # Calculate cutoff timestamp
    cutoff_time = datetime.fromisoformat(start_time) - timedelta(days=lookback_days)
    cutoff_str = cutoff_time.isoformat()

    # Build safe LIKE conditions
    conditions = []
    for substr in substrings:
        escaped = substr.strip().lower().replace("'", "''")
        conditions.append(f"LOWER(nogo_voice_name) LIKE '%{escaped}%'")

    name_condition = " OR ".join(conditions)

    query = f"""
    SELECT
        id,
        timestamp,
        user_uid,
        workspace_id,
        user_email,
        voice_id,
        nogo_voice_name,
        subscription_tier
    FROM `xi-labs.xi_prod.nogo_voice_check`
    WHERE timestamp >= TIMESTAMP('{cutoff_str}')
      AND ({name_condition})
    ORDER BY timestamp DESC
    """

    try:
        print("⏳ Running NGV attempt query...")
        ngv_df = client.query(query).to_dataframe()
        if ngv_df.empty:
            print("⚠️ No NGV attempts found.")
        else:
            print(f"✅ Retrieved {len(ngv_df)} NGV attempts.")
        return ngv_df
    except Exception as e:
        raise RuntimeError(f"ERROR during NGV attempt query: {e}")

# Run the query and store raw results
ngv_attempts_raw_df = get_ngv_attempts(INVESTIGATION_CONFIG, client)


⏳ Running NGV attempt query...
✅ Retrieved 37 NGV attempts.


In [4]:
# @title Cell 3: Filter NGV attempts by name substrings
import pandas as pd
import re

def filter_ngv_attempts_by_name_substrings(df: pd.DataFrame, substrings: list) -> pd.DataFrame:
    """
    Filters NGV attempts to only those where nogo_voice_name contains any of the given substrings.
    Substrings are case-insensitive and stripped of common punctuation.
    Also prints match counts per substring.
    """
    if df.empty:
        print("⚠️ No NGV attempts to filter.")
        return df

    # Clean input substrings
    clean_substrings = [s.strip().lower().replace("'", "").replace('"', "") for s in substrings]

    def matches_any_substring(name):
        if pd.isna(name):
            return False
        name_clean = str(name).lower().replace("'", "").replace('"', "")
        return any(sub in name_clean for sub in clean_substrings)

    # Filtered DataFrame
    filtered_df = df[df["nogo_voice_name"].apply(matches_any_substring)]

    # Per-substring match counts
    print(f"✅ {len(filtered_df)} NGV attempts matched voice names: {clean_substrings}")
    for substr in clean_substrings:
        count = df["nogo_voice_name"].dropna().apply(lambda x: substr in x.lower().replace("'", "").replace('"', "")).sum()
        print(f"   • {substr}: {count} matches")

    if filtered_df.empty:
        print("⚠️ No rows matched the input substrings.")

    return filtered_df

# Run it
ngv_attempts_df = filter_ngv_attempts_by_name_substrings(
    ngv_attempts_raw_df,
    INVESTIGATION_CONFIG["voice_names"]
)


✅ 37 NGV attempts matched voice names: ['obama']
   • obama: 37 matches


In [5]:
# @title NEW. ----CELL 4: Filter NGV attempts by fuzzy voice name match
import re

# Define filtering function
def filter_ngv_attempts_by_name_substrings(ngv_df, voice_substrings):
    if ngv_df.empty:
        print("⚠️ No NGV attempts to filter.")
        return ngv_df

    # Normalize substrings (e.g., escape apostrophes)
    clean_substrings = [re.escape(name.strip().lower()) for name in voice_substrings]

    # Match if any cleaned substring appears in the nogo_voice_name
    def matches_any_substring(name: str) -> bool:
        if not isinstance(name, str):
            return False
        name = name.lower()
        return any(substr in name for substr in clean_substrings)

    filtered_df = ngv_df[ngv_df["nogo_voice_name"].apply(matches_any_substring)]

    print(f"✅ Filtered NGV attempts: {len(filtered_df)} matched from {len(ngv_df)} total")
    return filtered_df

# Run it
ngv_attempts_df = filter_ngv_attempts_by_name_substrings(
    ngv_attempts_raw_df,
    INVESTIGATION_CONFIG["voice_names"]
)

# Preview
ngv_attempts_df.head()


✅ Filtered NGV attempts: 37 matched from 37 total


,id,timestamp,user_uid,workspace_id,user_email,voice_id,nogo_voice_name,subscription_tier
0,4WeOFe3MPxnrcS9hWz21,2025-07-17 02:00:03.417269+00:00,0w05uqMWl7bAxGDI8bV06xaQrPi2,fce16c0f663e41b9bda8b49d9981f443,eduardo.g.pedraza@gmail.com,gmsjqfQg3hrPABP3yUHD,Barack Obama,creator
1,451UOEuciKMmQQK4WpAI,2025-07-17 01:55:24.424666+00:00,0w05uqMWl7bAxGDI8bV06xaQrPi2,fce16c0f663e41b9bda8b49d9981f443,eduardo.g.pedraza@gmail.com,YJuUgZNre5Y4OXYi2xP1,Barack Obama,creator
2,KOqYZSD1ehFHa1ZJdH7K,2025-07-17 01:52:11.636265+00:00,0w05uqMWl7bAxGDI8bV06xaQrPi2,fce16c0f663e41b9bda8b49d9981f443,eduardo.g.pedraza@gmail.com,hZSM5oHTNrJ1HqSncoel,Barack Obama,creator
3,XOWAAIaXOQ3wBHUM34v2,2025-07-17 01:49:44.524747+00:00,0w05uqMWl7bAxGDI8bV06xaQrPi2,fce16c0f663e41b9bda8b49d9981f443,eduardo.g.pedraza@gmail.com,03K4UWXHgJCQLZl4OUyg,Barack Obama,creator
4,f686rv5UgeZA33dRNizH,2025-07-16 23:06:41.799618+00:00,GCW1C5XZybUhC9fmxlg3tqYuNt22,49ce0b5230f540bd88c0222a33161d82,dom@ghosteye.ai,jlDdyWW85FdtwUr0wBqn,Barack Obama,scale_2024_08_10


In [6]:
# @title CELL 5: Efficiently fetch all TTS generations for NGV users using a single BigQuery query

# ✅ CELL 5: Get TTS generations after NGV attempt (with audio_url for linking)

from datetime import timedelta

def get_tts_generations_after_ngv_window(ngv_attempts_df, client, window_minutes=30):
    """
    For each user NGV attempt, fetch TTS generations from the partitioned table
    within X minutes of their most recent NGV cloning attempt.
    """
    if ngv_attempts_df.empty:
        print("⚠️ No NGV attempts provided.")
        return []

    all_results = []

    for idx, row in ngv_attempts_df.iterrows():
        user_uid = row["user_uid"]
        timestamp = row["timestamp"]

        start_time = timestamp
        end_time = timestamp + timedelta(minutes=window_minutes)

        query = f"""
        SELECT
            user_uid,
            user_email,
            voice_name,
            text,
            timestamp,
            audio_url
        FROM `xi-labs.xi_prod.tts_usage_partitioned`
        WHERE user_uid = '{user_uid}'
        AND timestamp BETWEEN TIMESTAMP('{start_time}') AND TIMESTAMP('{end_time}')
        ORDER BY timestamp ASC
        """

        try:
            df = client.query(query).to_dataframe()
            all_results.append({
                "user_uid": user_uid,
                "ngv_row": row,
                "tts_generations": df
            })
            print(f"✅ {user_uid}: {len(df)} generations")
        except Exception as e:
            print(f"❌ {user_uid}: ERROR – {e}")

    return all_results

# Run the TTS pull
tts_results = get_tts_generations_after_ngv_window(
    ngv_attempts_df,
    client,
    window_minutes=INVESTIGATION_CONFIG["tts_window_minutes"]
)



✅ 0w05uqMWl7bAxGDI8bV06xaQrPi2: 14 generations
✅ 0w05uqMWl7bAxGDI8bV06xaQrPi2: 0 generations
✅ 0w05uqMWl7bAxGDI8bV06xaQrPi2: 0 generations
✅ 0w05uqMWl7bAxGDI8bV06xaQrPi2: 0 generations
✅ GCW1C5XZybUhC9fmxlg3tqYuNt22: 0 generations
✅ Pg0bjna7S1SgQdZIc7shoKKJRCB2: 4 generations
✅ bGOchIRw7uYJxBin7HeWpI09a2q1: 0 generations
✅ GCW1C5XZybUhC9fmxlg3tqYuNt22: 2 generations
✅ AVFkAjlQkNRemqBPATbwsFuIG4x1: 2 generations
✅ dOKl9umwZmQy1iq5S1IibhIb0LF2: 0 generations
✅ dOKl9umwZmQy1iq5S1IibhIb0LF2: 0 generations
✅ dOKl9umwZmQy1iq5S1IibhIb0LF2: 0 generations
✅ dOKl9umwZmQy1iq5S1IibhIb0LF2: 1 generations
✅ dOKl9umwZmQy1iq5S1IibhIb0LF2: 1 generations
✅ LpqLaAWW2ofDKPNAnUUcU8B3gFC3: 12 generations
✅ NrJNDutIEDg7HW3nNps4zF4ppMk2: 0 generations
✅ NrJNDutIEDg7HW3nNps4zF4ppMk2: 0 generations
✅ bK7o01Xkk2Q7CkT9y8CS42Ah7KQ2: 0 generations
✅ 466chYiBcbZaECRFe9WHZqW6MF22: 1 generations
✅ 466chYiBcbZaECRFe9WHZqW6MF22: 1 generations
✅ 466chYiBcbZaECRFe9WHZqW6MF22: 1 generations
✅ 466chYiBcbZaECRFe9WHZqW6MF22: 

In [7]:
# @title CELL 6: Stitch, translate, and render audio links with all metadata
# ✅ CELL 6: Stitch, translate, and render audio links with all metadata
import pandas as pd
from IPython.display import display, HTML
from tqdm.notebook import tqdm
import getpass
from openai import OpenAI

# 🔑 Prompt for OpenAI key
api_key = getpass.getpass("🔑 Enter your OpenAI API key:")
openai_client = OpenAI(api_key=api_key)

# 🌍 Translator with error handling
def translate_to_english(text):
    try:
        response = openai_client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "Translate the following text to English."},
                {"role": "user", "content": text}
            ],
            temperature=0.2
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"⚠️ Translation failed: {type(e).__name__} – {e}")
        return "TRANSLATION ERROR"

# 🔁 Narrative stitching + audio links
def stitch_ngv_tts_results_with_links(tts_results):
    if not tts_results:
        print("⚠️ No TTS results to summarize.")
        return pd.DataFrame()

    stitched_rows = []

    for result in tqdm(tts_results, desc="🔄 Stitching TTS narratives"):
        tts_df = result["tts_generations"]
        if tts_df.empty:
            continue

        user_uid = result["user_uid"]
        user_email = result["ngv_row"].get("user_email", "")
        workspace_id = result["ngv_row"].get("workspace_id", "")
        nogo_voice_name = result["ngv_row"].get("nogo_voice_name", "Unknown")

        grouped = tts_df.groupby("voice_name")

        for voice_name, group in grouped:
            group = group.sort_values("timestamp")
            full_text_block = " ".join(group["text"].astype(str).tolist())
            tts_count = len(group)

            # 🔊 Safe audio link construction
            if "audio_path" in group.columns:
                audio_links = group["audio_path"].dropna().unique()
                voice_files = " ".join(
                    [f'<a href="{link}" target="_blank">🔊</a>' for link in audio_links]
                )
            else:
                voice_files = "N/A"

            stitched_rows.append({
                "user_uid": user_uid,
                "user_email": user_email,
                "workspace_id": workspace_id,
                "nogo_voice_name": nogo_voice_name,
                "voice_name": voice_name,
                "tts_count": tts_count,
                "start_time": group["timestamp"].iloc[0],
                "end_time": group["timestamp"].iloc[-1],
                "full_text_block": full_text_block,
                "voice_files": voice_files
            })

    df = pd.DataFrame(stitched_rows)

    if df.empty:
        print("⚠️ No stitched narratives produced.")
        return df

    print(f"✅ Stitched {len(df)} narratives. Translating to English...")
    tqdm.pandas(desc="🌍 Translating")
    df["translated_text"] = df["full_text_block"].progress_apply(translate_to_english)
    return df

# 🚀 Run
tts_summary_df = stitch_ngv_tts_results_with_links(tts_results)

# 📋 Display with audio links
display(HTML(tts_summary_df[
    ["user_uid", "user_email", "workspace_id", "nogo_voice_name", "voice_name",
     "tts_count", "start_time", "end_time", "voice_files",
     "full_text_block", "translated_text"]
].to_html(escape=False)))


🔑 Enter your OpenAI API key:··········


🔄 Stitching TTS narratives:   0%|          | 0/37 [00:00<?, ?it/s]

✅ Stitched 21 narratives. Translating to English...


🌍 Translating:   0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
# @title Export Cell 7: Stitched and translated TTS results to CSV for download

import pandas as pd
from google.colab import files

# Define filename
export_filename = "tts_ngv_stitched_translated.csv"

# Save to CSV
tts_summary_df.to_csv(export_filename, index=False)
print(f"✅ File saved: {export_filename}")

# Trigger download
files.download(export_filename)


✅ File saved: tts_ngv_stitched_translated.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>